# Vulnerability Score Analysis

This notebook calculates **rainfall** and **heatwave** vulnerability scores per census section per day using:

1. **Data-driven weather variables** identified in `correlationAnalysis.ipynb`
2. **Socioeconomic factors** (IST index)
3. **Infrastructure factors** (leak frequency, density, consumption intensity)

The weather vulnerability components use recommended variables and weights from correlation analysis, ensuring evidence-based vulnerability assessment.

## 10. Save Results

Save the final dataset with vulnerability scores for use in mapping and analysis.


In [25]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json


## 1. Load Census Sections Data

Load the Barcelona census sections from CSV. The file contains polygon geometries in WKT format (WGS84 coordinate system).


In [26]:
# Load CSV with WKT geometry
df = gpd.read_file("data/BarcelonaCiutat_SeccionsCensals.csv", GEOM_POSSIBLE_NAMES="geometria_wgs84", KEEP_GEOM_COLUMNS="NO")

# Create GeoDataFrame with geometry
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Calculate centroids (for distance calculations later)
# Note: For accurate centroid calculations, we'll reproject to UTM in a later step
gdf["centroid"] = gdf.geometry.centroid

# Extract latitude and longitude from centroids
gdf["centroid_lat"] = gdf["centroid"].y
gdf["centroid_lon"] = gdf["centroid"].x

gdf.head()

/var/folders/gy/5nns71_n7nx_v_2mkns5_k340000gn/T/ipykernel_857/2562052214.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["centroid"] = gdf.geometry.centroid


,codi_districte,nom_districte,codi_barri,nom_barri,codi_aeb,codi_seccio_censal,geometria_etrs89,geometry,centroid,centroid_lat,centroid_lon
0,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...","POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219
1,01,Ciutat Vella,01,el Raval,001,002,"POLYGON ((431023.5455 4581164.3265, 430990.550...","POLYGON ((2.1751 41.37905, 2.1747 41.37951, 2....",POINT (2.17391 41.37793),41.377927,2.173914
2,01,Ciutat Vella,01,el Raval,002,003,"POLYGON ((430778.3455 4580930.5395, 430766.851...","POLYGON ((2.1722 41.37692, 2.17206 41.37696, 2...",POINT (2.17199 41.37576),41.375757,2.171985
3,01,Ciutat Vella,01,el Raval,002,004,"POLYGON ((430564.2645 4581104.2995, 430496.863...","POLYGON ((2.16962 41.37847, 2.16882 41.37784, ...",POINT (2.16924 41.37642),41.376416,2.169238
4,01,Ciutat Vella,01,el Raval,003,005,"POLYGON ((430905.0315 4581350.0725, 430874.963...","POLYGON ((2.17366 41.38071, 2.1733 41.38113, 2...",POINT (2.17277 41.37884),41.378836,2.172773


## 2. Create SECCIO_CENSAL Identifier

Create a unique identifier for each census section by concatenating:
- Prefix: `080193` (Barcelona municipality code)
- District code: `codi_districte`
- Section code: `codi_seccio_censal`

This creates a standardized identifier format: `080193 + codi_districte + codi_seccio_censal`


In [27]:
# Create SECCIO_CENSAL field: prefix 080193 + codi_districte + codi_seccio_censal
gdf["SECCIO_CENSAL"] = "080193" + gdf["codi_districte"].astype(str) + gdf["codi_seccio_censal"].astype(str)

# Verify the identifier creation
gdf[["codi_districte", "codi_seccio_censal", "SECCIO_CENSAL"]].head()


,codi_districte,codi_seccio_censal,SECCIO_CENSAL
0,01,001,08019301001
1,01,002,08019301002
2,01,003,08019301003
3,01,004,08019301004
4,01,005,08019301005


In [28]:
# Verify uniqueness of SECCIO_CENSAL
num_unique = gdf["SECCIO_CENSAL"].nunique()
print(f"Number of unique values in SECCIO_CENSAL: {num_unique}")
print(f"Total number of rows: {len(gdf)}")


Number of unique values in SECCIO_CENSAL: 1068
Total number of rows: 1068


**Note:** This corresponds to the number of census sections in Barcelona (1068), confirming that our identifier field is correct and unique for each section.

## 3. Load Weather Stations

Load the weather stations for which we have complete data for 2023 and 2024. 

**Note:** We exclude the Zoo weather station (X2) for our simplified version of the map, since it was dismantled in October 2024.

The three stations used are:
- **D5**: Located at coordinates (2.12379, 41.41864)
- **X4**: Located at coordinates (2.16775, 41.38390)
- **X8**: Located at coordinates (2.10540, 41.37919)


In [29]:
# Create GeoDataFrame with weather station locations
stations = gpd.GeoDataFrame(
    {
        "name": ["D5", "X4", "X8"],
        "lat": [41.41864, 41.38390, 41.37919],
        "lon": [2.12379, 2.16775, 2.10540],
    },
    geometry=gpd.points_from_xy(
        [2.12379, 2.16775, 2.10540],  # longitude (x)
        [41.41864, 41.38390, 41.37919],  # latitude (y)
    ),
    crs="EPSG:4326"  # WGS84 coordinate system
)

stations

,name,lat,lon,geometry
0,D5,41.41864,2.12379,POINT (2.12379 41.41864)
1,X4,41.38390,2.16775,POINT (2.16775 41.3839)
2,X8,41.37919,2.10540,POINT (2.1054 41.37919)


## 4. Assign Weather Stations to Census Sections

For each census section, we find the nearest weather station using spatial joins. This process involves:

1. **Reproject to UTM (EPSG:25831)**: Convert both datasets to a projected coordinate system (UTM Zone 31N) for accurate distance calculations in meters
2. **Calculate centroids**: Compute the centroid of each census section polygon in the projected CRS
3. **Find nearest station**: Use `sjoin_nearest` to find the closest weather station to each centroid
4. **Merge results**: Add the assigned station name and distance to the original GeoDataFrame


In [30]:
# Step 1: Reproject to UTM for accurate distance calculations
gdf_utm = gdf.to_crs(epsg=25831)  # UTM Zone 31N (Spain)
stations_utm = stations.to_crs(epsg=25831)

# Step 2: Calculate centroids in projected CRS (more accurate than WGS84)
gdf_utm["centroid"] = gdf_utm.geometry.centroid
centroids = gdf_utm.set_geometry("centroid")

# Step 3: Find nearest weather station to each census section centroid
nearest = gpd.sjoin_nearest(
    centroids,
    stations_utm[["name", "geometry"]],
    how="left",
    distance_col="dist_m"  # Distance in meters
)

# Step 4: Merge results back to original GeoDataFrame
# Merge on index since sjoin_nearest preserves the index from centroids
gdf = gdf.merge(
    nearest[["name", "dist_m"]],
    left_index=True,
    right_index=True,
    how="left"
)
gdf = gdf.rename(columns={"name": "WEATHER_STATION"})

# Display results
print(f"Number of census sections assigned to stations: {gdf['WEATHER_STATION'].notna().sum()}")
print(f"\nDistribution of stations:")
print(gdf["WEATHER_STATION"].value_counts())
print(f"\nDistance statistics (meters):")
print(gdf["dist_m"].describe())

Number of census sections assigned to stations: 1068

Distribution of stations:
WEATHER_STATION
X4    625
D5    338
X8    105
Name: count, dtype: int64

Distance statistics (meters):
count    1068.000000
mean     2988.696968
std      1492.095398
min       121.156567
25%      1914.985512
50%      2611.524009
75%      4095.279686
max      7152.228289
Name: dist_m, dtype: float64


## 5. Preview Results

Preview the final GeoDataFrame with assigned weather stations:


In [31]:
# Preview the final GeoDataFrame
gdf[["SECCIO_CENSAL", "nom_districte", "nom_barri", "WEATHER_STATION", "dist_m"]].head(10)

,SECCIO_CENSAL,nom_districte,nom_barri,WEATHER_STATION,dist_m
0,08019301001,Ciutat Vella,el Raval,X4,1325.662820
1,08019301002,Ciutat Vella,el Raval,X4,839.842507
2,08019301003,Ciutat Vella,el Raval,X4,970.927625
3,08019301004,Ciutat Vella,el Raval,X4,840.155800
4,08019301005,Ciutat Vella,el Raval,X4,701.768850
5,08019301006,Ciutat Vella,el Raval,X4,544.606912
6,08019301007,Ciutat Vella,el Raval,X4,560.631260
7,08019301008,Ciutat Vella,el Raval,X4,675.929543
8,08019301009,Ciutat Vella,el Raval,X4,726.465578
9,08019301010,Ciutat Vella,el Raval,X4,563.659021


## 6. Merge Daily Weather Data with Census Sections

Load the cleaned weather data and merge it with census sections. The weather data is in long format (one row per station-date-variable), so we need to pivot it to wide format (one row per station-date with columns for each variable).

### Date Range Filtering

For simplicity, we filter the weather data to the period from **2023-01-04 to 2024-12-31**. This represents the temporal overlap between all our data sources:
- **Weather data**: Available from 2021-01-01 to 2025-11-16
- **Consumption data**: Available from 2023-01-04 onwards
- **Leak incidents**: Available from 2023 onwards

By focusing on this common period, we ensure all data sources are available for analysis while maintaining a substantial time range for our vulnerability score calculations.


In [32]:
# Load weather data and filter to only the stations we use (D5, X4, X8)
weather = pd.read_parquet("clean/weather_clean.parquet")

# Filter to only the stations assigned to census sections
stations_to_keep = ['D5', 'X4', 'X8']
weather = weather[weather['CODI_ESTACIO'].isin(stations_to_keep)].copy()

# Drop NOM_ESTACIO since we already have CODI_ESTACIO
if 'NOM_ESTACIO' in weather.columns:
    weather = weather.drop(columns=['NOM_ESTACIO'])

# Filter to date range: 2023-01-04 to 2024-12-31 (temporal overlap with consumption and leaks)
date_start = pd.to_datetime('2023-01-04')
date_end = pd.to_datetime('2024-12-31')
weather = weather[
    (pd.to_datetime(weather['DATA_LECTURA']) >= date_start) & 
    (pd.to_datetime(weather['DATA_LECTURA']) <= date_end)
].copy()

print(f"Weather data shape (after filtering): {weather.shape}")
print(f"\nStations in weather data: {sorted(weather['CODI_ESTACIO'].unique())}")
print(f"\nDate range: {weather['DATA_LECTURA'].min()} to {weather['DATA_LECTURA'].max()}")
print(f"\nNumber of variables: {weather['NOM_VARIABLE'].nunique()}")
print(f"\nSample of weather data:")
weather.head()


Weather data shape (after filtering): (48021, 9)

Stations in weather data: ['D5', 'X4', 'X8']

Date range: 2023-01-04 00:00:00 to 2024-12-31 00:00:00

Number of variables: 22

Sample of weather data:


,ID,CODI_ESTACIO,DATA_LECTURA,CODI_VARIABLE,NOM_VARIABLE,VALOR,UNITAT,HORA _TU,VALOR_NUM
54232,D51000012304,D5,2023-01-04,1.000,Temperatura mitjana diària,"11,7",°C,NA,11.7
54233,D51001012304,D5,2023-01-04,1.001,Temperatura màxima diària + hora,"17,7",°C,14:14:00,17.7
54234,D51002012304,D5,2023-01-04,1.002,Temperatura mínima diària + hora,"9,1",°C,06:06:00,9.1
54235,D51003012304,D5,2023-01-04,1.003,Temperatura mitjana diària clàssica,"13,4",°C,NA,13.4
54236,D51004012304,D5,2023-01-04,1.004,Amplitud tèrmica diària,"8,6",°C,NA,8.6


In [33]:
# Pivot weather data from long to wide format
# Each row will be a unique combination of station (CODI_ESTACIO) and date (DATA_LECTURA)
# Each variable (NOM_VARIABLE) becomes a column with its VALOR_NUM value
# Note: No duplicate measurements - each station-date-variable has only one value

weather_daily = weather.pivot_table(
    index=['CODI_ESTACIO', 'DATA_LECTURA'],
    columns='NOM_VARIABLE',
    values='VALOR_NUM',
    aggfunc='first'  # Since there are no duplicates, 'first' is sufficient
).reset_index()

# Flatten column names (remove multi-index if any)
weather_daily.columns.name = None

print(f"Weather daily shape: {weather_daily.shape}")
print(f"Number of station-date combinations: {len(weather_daily)}")
print(f"Number of weather variables: {len(weather_daily.columns)}")
print(f"\nColumns: {list(weather_daily.columns[:10])}...")  # Show first 10 columns
weather_daily.head()


Weather daily shape: (2184, 24)
Number of station-date combinations: 2184
Number of weather variables: 24

Columns: ['CODI_ESTACIO', 'DATA_LECTURA', 'Amplitud tèrmica diària', 'Direcció de la ratxa màx. diària de vent 10 m', 'Direcció mitjana diària del vent 10 m (m. 1)', 'Evapotranspiració de referència', 'Humitat relativa mitjana diària', 'Humitat relativa màxima diària + data', 'Humitat relativa mínima diària + data', 'Irradiació solar global diària']...


,CODI_ESTACIO,DATA_LECTURA,Amplitud tèrmica diària,Direcció de la ratxa màx. diària de vent 10 m,Direcció mitjana diària del vent 10 m (m. 1),Evapotranspiració de referència,Humitat relativa mitjana diària,Humitat relativa màxima diària + data,Humitat relativa mínima diària + data,Irradiació solar global diària,...,Precipitació màxima en 30 min (diària)+ hora,Pressió atmosfèrica mitjana diària,Pressió atmosfèrica màxima diària + hora,Pressió atmosfèrica mínima diària + hora,Ratxa màxima diària del vent 10 m + hora,Temperatura mitjana diària,Temperatura mitjana diària clàssica,Temperatura màxima diària + hora,Temperatura mínima diària + hora,Velocitat mitjana diària del vent 10 m (esc.)
0,D5,2023-01-04,8.6,338.0,335.0,1.09,66.0,87.0,39.0,8.9,...,0.0,983.1,984.7,982.3,7.6,11.7,13.4,17.7,9.1,3.5
1,D5,2023-01-05,7.6,304.0,263.0,1.18,53.0,67.0,35.0,9.2,...,0.0,979.0,982.7,975.8,8.7,11.7,12.5,16.3,8.7,3.5
2,D5,2023-01-06,7.7,310.0,284.0,1.02,59.0,77.0,47.0,9.2,...,0.0,975.5,976.9,974.7,10.4,10.0,11.1,14.9,7.2,3.9
3,D5,2023-01-07,6.1,295.0,267.0,1.13,67.0,100.0,46.0,9.6,...,0.0,971.3,974.7,968.7,14.7,9.5,10.3,13.3,7.2,5.8
4,D5,2023-01-08,3.8,274.0,266.0,0.58,72.0,86.0,56.0,3.2,...,0.1,964.4,968.8,961.1,11.9,11.9,11.7,13.6,9.8,4.5


In [34]:
# Create cartesian product: one row per census section per date
# This ensures we have complete daily structure (one row per census section per day)

# Get unique dates from weather data
date_start = pd.to_datetime('2023-01-04')
date_end = pd.to_datetime('2024-12-31')
date_range = pd.date_range(start=date_start, end=date_end, freq='D')
dates_df = pd.DataFrame({'DATA_LECTURA': date_range})

# Create cartesian product: census sections × dates
# Select only necessary columns from gdf (excluding geometry temporarily)
gdf_stations = gdf[['SECCIO_CENSAL', 'WEATHER_STATION'] + 
                   [col for col in gdf.columns 
                    if col not in ['SECCIO_CENSAL', 'WEATHER_STATION', 'geometry', 'centroid', 'centroid_lat', 'centroid_lon', 'dist_m']]].copy()

# Create cartesian product using merge with dummy key
gdf_stations['key'] = 1
dates_df['key'] = 1
gdf_daily = gdf_stations.merge(dates_df, on='key').drop(columns='key')

# Now merge weather data by matching WEATHER_STATION with CODI_ESTACIO and matching dates
gdf_daily = gdf_daily.merge(
    weather_daily,
    left_on=['WEATHER_STATION', 'DATA_LECTURA'],
    right_on=['CODI_ESTACIO', 'DATA_LECTURA'],
    how='left'
)

# Drop WEATHER_STATION since it's the same as CODI_ESTACIO (which comes from weather data)
gdf_daily = gdf_daily.drop(columns=['WEATHER_STATION'])

# Merge back geometry and other spatial info from original gdf
gdf_spatial = gdf[['SECCIO_CENSAL', 'geometry', 'centroid', 'centroid_lat', 'centroid_lon', 'dist_m']].copy()
gdf_daily = gdf_daily.merge(gdf_spatial, on='SECCIO_CENSAL', how='left')

# Convert back to GeoDataFrame
gdf_daily = gpd.GeoDataFrame(gdf_daily, geometry='geometry', crs='EPSG:4326')

# Verify structure
expected_rows = len(gdf['SECCIO_CENSAL'].unique()) * len(date_range)
actual_rows = len(gdf_daily)

print(f"Final GeoDataFrame shape: {gdf_daily.shape}")
print(f"\nNumber of census sections: {gdf_daily['SECCIO_CENSAL'].nunique()}")
print(f"Number of unique dates: {gdf_daily['DATA_LECTURA'].nunique()}")
print(f"Expected rows (sections × dates): {expected_rows:,}")
print(f"Actual rows: {actual_rows:,}")
print(f"Match: {'✓' if actual_rows == expected_rows else '✗'}")
print(f"\nDate range: {gdf_daily['DATA_LECTURA'].min()} to {gdf_daily['DATA_LECTURA'].max()}")

# Show sample
gdf_daily.head(10)


Final GeoDataFrame shape: (777504, 37)

Number of census sections: 1068
Number of unique dates: 728
Expected rows (sections × dates): 777,504
Actual rows: 777,504
Match: ✓

Date range: 2023-01-04 00:00:00 to 2024-12-31 00:00:00


,SECCIO_CENSAL,codi_districte,nom_districte,codi_barri,nom_barri,codi_aeb,codi_seccio_censal,geometria_etrs89,DATA_LECTURA,CODI_ESTACIO,...,Temperatura mitjana diària,Temperatura mitjana diària clàssica,Temperatura màxima diària + hora,Temperatura mínima diària + hora,Velocitat mitjana diària del vent 10 m (esc.),geometry,centroid,centroid_lat,centroid_lon,dist_m
0,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-04,X4,...,13.0,13.5,16.8,10.2,0.9,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282
1,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-05,X4,...,12.5,13.1,17.3,8.8,1.1,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282
2,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-06,X4,...,11.8,11.9,16.0,7.7,0.9,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282
3,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-07,X4,...,12.5,13.0,16.7,9.3,1.7,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282
4,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-08,X4,...,14.2,14.0,16.2,11.7,1.8,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282
5,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-09,X4,...,14.5,14.4,17.1,11.7,4.2,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282
6,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-10,X4,...,12.2,12.6,15.7,9.4,1.6,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282
7,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-11,X4,...,13.0,13.2,16.4,9.9,1.2,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282
8,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-12,X4,...,12.2,12.8,16.0,9.6,1.9,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282
9,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-13,X4,...,12.7,13.5,16.8,10.2,1.1,"POLYGON ((2.17575 41.37827, 2.17552 41.37865, ...",POINT (2.17722 41.37432),41.374324,2.177219,1325.66282


In [35]:
# Prepare weather columns list for vulnerability calculation
# Get all weather variable columns (exclude non-weather columns)
non_weather_cols = ['SECCIO_CENSAL', 'DATA_LECTURA', 'CODI_ESTACIO', 'geometry', 'centroid', 
                    'centroid_lat', 'centroid_lon', 'dist_m', 'ist', 'CONSUMO_TOTAL', 'NUM_FUITES',
                    'codi_districte', 'nom_districte', 'codi_barri', 'nom_barri', 'codi_aeb', 
                    'codi_seccio_censal', 'NUM_CONTRACTS', 'CONSUMO_POR_CONTRATO', 'CONSUMO_PROMEDIO', 
                    'CONSUMO_MEDIANA', 'NUM_CONTRATOS_CON_FUITES', 'FUITES_MAX_POR_CONTRATO', 
                    'FUITES_PROMEDIO_POR_CONTRATO', 'DATA_LECTURA_dt', 'month']

# Get all columns that are weather variables (from the pivot operation)
# These are columns that aren't in the non-weather list
weather_cols = [col for col in gdf_daily.columns if col not in non_weather_cols]

# Filter to only columns that contain numeric weather data (exclude text/object columns)
weather_cols = [col for col in weather_cols if gdf_daily[col].dtype in ['float64', 'int64', 'float32', 'int32']]

print(f"Found {len(weather_cols)} weather variable columns")
print(f"Sample weather columns: {weather_cols[:10]}")
print(f"\nAll weather columns:")
for col in sorted(weather_cols):
    print(f"  - {col}")


Found 22 weather variable columns
Sample weather columns: ['Amplitud tèrmica diària', 'Direcció de la ratxa màx. diària de vent 10 m', 'Direcció mitjana diària del vent 10 m (m. 1)', 'Evapotranspiració de referència', 'Humitat relativa mitjana diària', 'Humitat relativa màxima diària + data', 'Humitat relativa mínima diària + data', 'Irradiació solar global diària', 'Precipitació acumulada diària', 'Precipitació acumulada diària (8-8 h)']

All weather columns:
  - Amplitud tèrmica diària
  - Direcció de la ratxa màx. diària de vent 10 m
  - Direcció mitjana diària del vent 10 m (m. 1)
  - Evapotranspiració de referència
  - Humitat relativa mitjana diària
  - Humitat relativa màxima diària + data
  - Humitat relativa mínima diària + data
  - Irradiació solar global diària
  - Precipitació acumulada diària
  - Precipitació acumulada diària (8-8 h)
  - Precipitació màxima en 1 h (diària) + hora
  - Precipitació màxima en 1 min (diària) + hora
  - Precipitació màxima en 30 min (diària)+ h

### Current Structure of `gdf_daily`

At this point, `gdf_daily` contains:

**Rows (Observations):**
- One row per **census section** per **date** (from 2023-01-04 to 2024-12-31)
- Total rows = Number of census sections (1068) × Number of dates in the filtered period
- Each row represents a unique combination of a census section and a date

**Columns (Variables):**

1. **Census Section Information:**
   - `SECCIO_CENSAL`: Unique identifier (format: 080193 + district + section)
   - `codi_districte`, `nom_districte`: District codes and names
   - `codi_barri`, `nom_barri`: Neighborhood codes and names
   - `codi_aeb`, `codi_seccio_censal`: Additional identifiers
   - `geometry`: Polygon geometry of the census section
   - `centroid`, `centroid_lat`, `centroid_lon`: Geographic centroids

2. **Weather Station Assignment:**
   - `CODI_ESTACIO`: Assigned weather station code (D5, X4, or X8)
   - `dist_m`: Distance to nearest weather station (in meters)

3. **Weather Variables (24 columns):**
   - `DATA_LECTURA`: Date of the weather reading
   - All 24 weather variables from the weather stations (temperature, precipitation, humidity, pressure, wind, etc.)

**Next Steps:**
We will now merge consumption, leak incidents, and socioeconomic data to enrich this dataset further.


## 7. Merge Additional Data Sources

Now we'll merge consumption, leak incidents, and socioeconomic data with the weather-enriched census sections. Each dataset needs to be aggregated appropriately to match the daily structure of `gdf_daily`.


### 7.1 Load and Merge Socioeconomic Data (IST)

The socioeconomic data (IST - Índex socioeconòmic territorial) is a static factor that we will keep constant across years (for now) for each census section. We'll load it and merge directly by `SECCIO_CENSAL`.


In [36]:
# Load socioeconomic data
socio = pd.read_parquet("clean/socio_clean.parquet")

print(f"Socioeconomic data shape: {socio.shape}")
print(f"Years: {sorted(socio['any'].unique())}")
print(f"Number of unique census sections: {socio['SECCIO_CENSAL'].nunique()}")
print(f"\nSample socioeconomic data:")
socio.head()


Socioeconomic data shape: (1068, 4)
Years: [2022]
Number of unique census sections: 1068

Sample socioeconomic data:


,any,SECCIO_CENSAL,concepte,valor
0,2022,08019301001,Índex socioeconòmic territorial,85.7
1,2022,08019301002,Índex socioeconòmic territorial,75.8
2,2022,08019301003,Índex socioeconòmic territorial,73.7
3,2022,08019301004,Índex socioeconòmic territorial,81.8
4,2022,08019301005,Índex socioeconòmic territorial,79.1


In [37]:
# Since IST is static (constant across years), we'll take one value per SECCIO_CENSAL
# Filter to get the IST value (concepte = "Índex socioeconòmic territorial")
socio_ist = socio[socio['concepte'] == 'Índex socioeconòmic territorial'].copy()

# Select SECCIO_CENSAL and valor, rename valor to ist
socio_ist = socio_ist[['SECCIO_CENSAL', 'valor']].copy()
socio_ist = socio_ist.rename(columns={'valor': 'ist'})

# If there are multiple years, take the most recent one (or first if all same)
# Group by SECCIO_CENSAL and take the first value (they should all be the same anyway)
socio_ist = socio_ist.groupby('SECCIO_CENSAL')['ist'].first().reset_index()

print(f"Socioeconomic IST shape: {socio_ist.shape}")
print(f"Number of unique census sections: {socio_ist['SECCIO_CENSAL'].nunique()}")
print(f"\nIST statistics:")
print(socio_ist['ist'].describe())
print(f"\nSample IST data:")
socio_ist.head()


Socioeconomic IST shape: (1068, 2)
Number of unique census sections: 1068

IST statistics:
count    1068.000000
mean      108.917603
std        14.578281
min        51.700000
25%       101.200000
50%       111.100000
75%       118.200000
max       138.000000
Name: ist, dtype: float64

Sample IST data:


,SECCIO_CENSAL,ist
0,08019301001,85.7
1,08019301002,75.8
2,08019301003,73.7
3,08019301004,81.8
4,08019301005,79.1


### 7.2 Load and Aggregate Consumption Data

Consumption data is split across multiple parquet files. We'll load all files, aggregate by `SECCIO_CENSAL` and `FECHA` (date), and calculate daily consumption metrics.


In [38]:
import glob
import os

# Load all consumption parquet files
consum_files = glob.glob("clean/split_consum_bcn/consum_clean_bcn_part_*.parquet")
consum_files.sort()  # Ensure consistent order

print(f"Found {len(consum_files)} consumption files")

# Load and concatenate all consumption files
consum_list = []
for file in consum_files:
    df = pd.read_parquet(file)
    consum_list.append(df)

consum = pd.concat(consum_list, ignore_index=True)

# Filter to start at 2023-01-04 (temporal overlap with weather and leaks)
date_start = pd.to_datetime('2023-01-04')
consum = consum[pd.to_datetime(consum['FECHA']) >= date_start].copy()

print(f"\nTotal consumption records (after filtering): {len(consum)}")
print(f"Date range: {consum['FECHA'].min()} to {consum['FECHA'].max()}")
print(f"Number of unique census sections: {consum['SECCIO_CENSAL'].nunique()}")
print(f"\nSample consumption data:")
consum.head()


Found 18 consumption files

Total consumption records (after filtering): 5014350
Date range: 2023-01-04 00:00:00 to 2024-12-31 00:00:00
Number of unique census sections: 621

Sample consumption data:


,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,US_AIGUA_GEST,DATA_INST_COMP
730,VECWAVDUULZDSBOP,2023-01-04,2070,08019303025,C,2016-04-25
731,VECWAVDUULZDSBOP,2023-01-05,1938,08019303025,C,2016-04-25
732,VECWAVDUULZDSBOP,2023-01-06,4,08019303025,C,2016-04-25
733,VECWAVDUULZDSBOP,2023-01-07,53,08019303025,C,2016-04-25
734,VECWAVDUULZDSBOP,2023-01-08,7,08019303025,C,2016-04-25


In [39]:
# Aggregate consumption with multiple metrics to capture different aspects:
# 1. Total consumption (demand level)
# 2. Number of contracts/meters (infrastructure density)
# 3. Average consumption per contract (intensity per meter)
# 4. Median consumption per contract (robust to outliers)
consum_daily = consum.groupby(['SECCIO_CENSAL', 'FECHA'], as_index=False).agg(
    CONSUMO_TOTAL=('CONSUMO_REAL', 'sum'),  # Total consumption (demand)
    NUM_CONTRACTS=('POLIZA_SUMINISTRO', 'nunique'),  # Number of unique contracts/meters
    CONSUMO_PROMEDIO=('CONSUMO_REAL', 'mean'),  # Average consumption per contract
    CONSUMO_MEDIANA=('CONSUMO_REAL', 'median'),  # Median consumption per contract (robust)
)

# Calculate consumption per contract (intensity metric)
consum_daily['CONSUMO_POR_CONTRATO'] = consum_daily['CONSUMO_TOTAL'] / consum_daily['NUM_CONTRACTS'].replace(0, np.nan)

# Rename FECHA for consistency
consum_daily = consum_daily.rename(columns={'FECHA': 'DATA_LECTURA'})

print(f"Consumption daily shape: {consum_daily.shape}")
print(f"Date range: {consum_daily['DATA_LECTURA'].min()} to {consum_daily['DATA_LECTURA'].max()}")
print(f"\nConsumption metrics summary:")
print(f"  Total consumption range: {consum_daily['CONSUMO_TOTAL'].min():.0f} - {consum_daily['CONSUMO_TOTAL'].max():.0f} L/day")
print(f"  Contracts per section range: {consum_daily['NUM_CONTRACTS'].min():.0f} - {consum_daily['NUM_CONTRACTS'].max():.0f}")
print(f"  Consumption per contract range: {consum_daily['CONSUMO_POR_CONTRATO'].min():.2f} - {consum_daily['CONSUMO_POR_CONTRATO'].max():.2f} L/day/contract")
consum_daily.head()


Consumption daily shape: (450195, 7)
Date range: 2023-01-04 00:00:00 to 2024-12-31 00:00:00

Consumption metrics summary:
  Total consumption range: 0 - 466910 L/day
  Contracts per section range: 1 - 256
  Consumption per contract range: 0.00 - 55678.00 L/day/contract


,SECCIO_CENSAL,DATA_LECTURA,CONSUMO_TOTAL,NUM_CONTRACTS,CONSUMO_PROMEDIO,CONSUMO_MEDIANA,CONSUMO_POR_CONTRATO
0,08019301001,2023-01-04,4948,15,329.866667,330.0,329.866667
1,08019301001,2023-01-05,5259,15,350.600000,338.0,350.600000
2,08019301001,2023-01-06,5006,15,333.733333,313.0,333.733333
3,08019301001,2023-01-07,6301,15,420.066667,416.0,420.066667
4,08019301001,2023-01-08,5428,15,361.866667,380.0,361.866667


### 7.3 Load and Aggregate Leak Incidents Data

In [40]:
# Load leak incidents data
leaks = pd.read_parquet("clean/fuites_clean_bcn.parquet")

# Filter to date range: 2023-01-04 to 2024-12-31 (temporal overlap with weather and consumption)
date_start = pd.to_datetime('2023-01-04')
date_end = pd.to_datetime('2024-12-31')
leaks = leaks[
    (pd.to_datetime(leaks['CREATED_MENSAJE']) >= date_start) & 
    (pd.to_datetime(leaks['CREATED_MENSAJE']) <= date_end)
].copy()

print(f"Leak incidents shape (after filtering): {leaks.shape}")
print(f"Date range: {leaks['CREATED_MENSAJE'].min()} to {leaks['CREATED_MENSAJE'].max()}")
print(f"Number of unique census sections: {leaks['SECCIO_CENSAL'].nunique()}")

# Note: Not all dates will have leaks - this is normal
# When merged with gdf_daily, days without leaks will have NUM_FUITES = 0

print(f"\nSample leak data:")
leaks.head()


Leak incidents shape (after filtering): (1243, 5)
Date range: 2023-01-04 to 2024-12-30
Number of unique census sections: 428

Sample leak data:


,POLISSA_SUBM,CREATED_MENSAJE,CODIGO_MENSAJE,US_AIGUA_SUBM,SECCIO_CENSAL
0,KWHZ5UG2ZKENUFC2,2023-12-03,FUITA,DOMÈSTIC,08019305059
1,GVXPU34GVXQUIWFK,2023-08-10,FUITA,DOMÈSTIC,08019310139
2,GVXPU34GVXQUIWFK,2023-06-10,FUITA,DOMÈSTIC,08019310139
3,I7GGTJ6C6FMR5ARW,2024-09-06,FUITA,DOMÈSTIC,08019302087
4,I7GGTJ6C6FMR5ARW,2024-11-13,FUITA,DOMÈSTIC,08019302087


In [41]:
# Convert CREATED_MENSAJE to datetime if needed
leaks['CREATED_MENSAJE'] = pd.to_datetime(leaks['CREATED_MENSAJE'])

# Aggregate leak incidents with multiple metrics to capture different aspects:
# 1. Total number of leak incidents (NUM_FUITES) - overall leak count
# 2. Number of unique contracts with leaks (NUM_CONTRATOS_CON_FUITES) - leak density
# 3. Maximum leaks from a single contract (FUITES_MAX_POR_CONTRATO) - indicates severity
# 4. Average leaks per contract (FUITES_PROMEDIO_POR_CONTRATO) - intensity metric

# First, count leaks per contract per day to get contract-level metrics
leaks_by_contract = leaks.groupby(['SECCIO_CENSAL', 'CREATED_MENSAJE', 'POLISSA_SUBM'], as_index=False).agg(
    LEAKS_PER_CONTRACT=('POLISSA_SUBM', 'count')  # Count leaks per contract per day
)

# Then aggregate to census section level with multiple metrics
leaks_daily = leaks.groupby(['SECCIO_CENSAL', 'CREATED_MENSAJE'], as_index=False).agg(
    NUM_FUITES=('POLISSA_SUBM', 'count'),  # Total leak incidents (rows)
    NUM_CONTRATOS_CON_FUITES=('POLISSA_SUBM', 'nunique'),  # Number of unique contracts with leaks
)

# Merge with contract-level metrics to get max leaks per contract
max_leaks_per_contract = leaks_by_contract.groupby(['SECCIO_CENSAL', 'CREATED_MENSAJE'], as_index=False).agg(
    FUITES_MAX_POR_CONTRATO=('LEAKS_PER_CONTRACT', 'max')  # Maximum leaks from any single contract
)

leaks_daily = leaks_daily.merge(max_leaks_per_contract, on=['SECCIO_CENSAL', 'CREATED_MENSAJE'], how='left')

# Calculate average leaks per contract (intensity metric)
leaks_daily['FUITES_PROMEDIO_POR_CONTRATO'] = leaks_daily['NUM_FUITES'] / leaks_daily['NUM_CONTRATOS_CON_FUITES'].replace(0, np.nan)

# Fill NaN values for days with no leaks
leaks_daily['NUM_CONTRATOS_CON_FUITES'] = leaks_daily['NUM_CONTRATOS_CON_FUITES'].fillna(0).astype(int)
leaks_daily['FUITES_MAX_POR_CONTRATO'] = leaks_daily['FUITES_MAX_POR_CONTRATO'].fillna(0).astype(int)
leaks_daily['FUITES_PROMEDIO_POR_CONTRATO'] = leaks_daily['FUITES_PROMEDIO_POR_CONTRATO'].fillna(0)

# Rename for consistency
leaks_daily = leaks_daily.rename(columns={'CREATED_MENSAJE': 'DATA_LECTURA'})

print(f"Leaks daily shape: {leaks_daily.shape}")
print(f"Date range: {leaks_daily['DATA_LECTURA'].min()} to {leaks_daily['DATA_LECTURA'].max()}")
print(f"\nLeak metrics summary:")
print(f"  Total leaks per day range: {leaks_daily['NUM_FUITES'].min():.0f} - {leaks_daily['NUM_FUITES'].max():.0f}")
print(f"  Contracts with leaks per day range: {leaks_daily['NUM_CONTRATOS_CON_FUITES'].min():.0f} - {leaks_daily['NUM_CONTRATOS_CON_FUITES'].max():.0f}")
print(f"  Max leaks per contract range: {leaks_daily['FUITES_MAX_POR_CONTRATO'].min():.0f} - {leaks_daily['FUITES_MAX_POR_CONTRATO'].max():.0f}")
print(f"  Average leaks per contract range: {leaks_daily['FUITES_PROMEDIO_POR_CONTRATO'].min():.2f} - {leaks_daily['FUITES_PROMEDIO_POR_CONTRATO'].max():.2f}")
print(f"\nExample: Days with multiple leaks from different contracts:")
print(leaks_daily[leaks_daily['NUM_FUITES'] > 1].head())
leaks_daily.head()

Leaks daily shape: (1235, 6)
Date range: 2023-01-04 00:00:00 to 2024-12-30 00:00:00

Leak metrics summary:
  Total leaks per day range: 1 - 2
  Contracts with leaks per day range: 1 - 2
  Max leaks per contract range: 1 - 1
  Average leaks per contract range: 1.00 - 1.00

Example: Days with multiple leaks from different contracts:
    SECCIO_CENSAL DATA_LECTURA  NUM_FUITES  NUM_CONTRATOS_CON_FUITES  \
44    08019301020   2024-05-07           2                         2   
55    08019301021   2023-11-08           2                         2   
56    08019301021   2024-01-10           2                         2   
465   08019303025   2024-12-18           2                         2   
685   08019305025   2024-07-18           2                         2   

     FUITES_MAX_POR_CONTRATO  FUITES_PROMEDIO_POR_CONTRATO  
44                         1                           1.0  
55                         1                           1.0  
56                         1                       

,SECCIO_CENSAL,DATA_LECTURA,NUM_FUITES,NUM_CONTRATOS_CON_FUITES,FUITES_MAX_POR_CONTRATO,FUITES_PROMEDIO_POR_CONTRATO
0,08019301002,2024-06-19,1,1,1,1.0
1,08019301002,2024-08-16,1,1,1,1.0
2,08019301004,2023-03-22,1,1,1,1.0
3,08019301004,2024-03-04,1,1,1,1.0
4,08019301004,2024-06-25,1,1,1,1.0


### 7.4 Merge All Data Sources

Now we'll merge consumption, leaks, and socioeconomic data with the weather-enriched `gdf_daily` GeoDataFrame.


In [42]:
# Step 1: Merge socioeconomic data (IST) - static factor
# Merge on SECCIO_CENSAL only (no date needed since it's constant)
gdf_daily = gdf_daily.merge(
    socio_ist,
    on='SECCIO_CENSAL',
    how='left'
)

print(f"After IST merge: {gdf_daily.shape}")
print(f"IST records matched: {gdf_daily['ist'].notna().sum()}")

# Step 2: Merge consumption data
# Merge on SECCIO_CENSAL and DATA_LECTURA (date)
gdf_daily = gdf_daily.merge(
    consum_daily,
    on=['SECCIO_CENSAL', 'DATA_LECTURA'],
    how='left'
)

# Note: Consumption metrics may be NaN for dates without consumption data
# This is expected and will be handled in the vulnerability calculation
# We keep NaN values (not filling with 0) since missing consumption data is meaningful

print(f"\nAfter consumption merge: {gdf_daily.shape}")
print(f"Consumption records matched: {gdf_daily['CONSUMO_TOTAL'].notna().sum()}")
consum_metrics = ['CONSUMO_TOTAL', 'NUM_CONTRACTS', 'CONSUMO_POR_CONTRATO', 'CONSUMO_PROMEDIO', 'CONSUMO_MEDIANA']
consum_metrics_found = [m for m in consum_metrics if m in gdf_daily.columns]
print(f"Consumption metrics available: {', '.join(consum_metrics_found)}")

# Step 3: Merge leak incidents data
# Merge on SECCIO_CENSAL and DATA_LECTURA (date)
gdf_daily = gdf_daily.merge(
    leaks_daily,
    on=['SECCIO_CENSAL', 'DATA_LECTURA'],
    how='left'
)

# Fill NaN with 0 for all leak metrics (no leaks = 0)
# Using all aggregation metrics from section 7.3
leak_metrics = ['NUM_FUITES', 'NUM_CONTRATOS_CON_FUITES', 'FUITES_MAX_POR_CONTRATO', 'FUITES_PROMEDIO_POR_CONTRATO']
for metric in leak_metrics:
    if metric in gdf_daily.columns:
        if metric in ['NUM_FUITES', 'NUM_CONTRATOS_CON_FUITES', 'FUITES_MAX_POR_CONTRATO']:
            gdf_daily[metric] = gdf_daily[metric].fillna(0).astype(int)
        else:  # FUITES_PROMEDIO_POR_CONTRATO (float)
            gdf_daily[metric] = gdf_daily[metric].fillna(0.0)

print(f"\nAfter leaks merge: {gdf_daily.shape}")
print(f"Days with leaks: {(gdf_daily['NUM_FUITES'] > 0).sum()}")
print(f"Leak metrics initialized: {', '.join([m for m in leak_metrics if m in gdf_daily.columns])}")

print(f"\nFinal columns: {len(gdf_daily.columns)}")
print(f"\nSample of final merged data:")
gdf_daily.head(10)


After IST merge: (777504, 38)
IST records matched: 777504

After consumption merge: (777504, 43)
Consumption records matched: 450195
Consumption metrics available: CONSUMO_TOTAL, NUM_CONTRACTS, CONSUMO_POR_CONTRATO, CONSUMO_PROMEDIO, CONSUMO_MEDIANA

After leaks merge: (777504, 47)
Days with leaks: 1235
Leak metrics initialized: NUM_FUITES, NUM_CONTRATOS_CON_FUITES, FUITES_MAX_POR_CONTRATO, FUITES_PROMEDIO_POR_CONTRATO

Final columns: 47

Sample of final merged data:


,SECCIO_CENSAL,codi_districte,nom_districte,codi_barri,nom_barri,codi_aeb,codi_seccio_censal,geometria_etrs89,DATA_LECTURA,CODI_ESTACIO,...,ist,CONSUMO_TOTAL,NUM_CONTRACTS,CONSUMO_PROMEDIO,CONSUMO_MEDIANA,CONSUMO_POR_CONTRATO,NUM_FUITES,NUM_CONTRATOS_CON_FUITES,FUITES_MAX_POR_CONTRATO,FUITES_PROMEDIO_POR_CONTRATO
0,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-04,X4,...,85.7,4948.0,15.0,329.866667,330.0,329.866667,0,0,0,0.0
1,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-05,X4,...,85.7,5259.0,15.0,350.600000,338.0,350.600000,0,0,0,0.0
2,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-06,X4,...,85.7,5006.0,15.0,333.733333,313.0,333.733333,0,0,0,0.0
3,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-07,X4,...,85.7,6301.0,15.0,420.066667,416.0,420.066667,0,0,0,0.0
4,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-08,X4,...,85.7,5428.0,15.0,361.866667,380.0,361.866667,0,0,0,0.0
5,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-09,X4,...,85.7,4970.0,15.0,331.333333,368.0,331.333333,0,0,0,0.0
6,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-10,X4,...,85.7,4487.0,15.0,299.133333,330.0,299.133333,0,0,0,0.0
7,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-11,X4,...,85.7,4972.0,15.0,331.466667,292.0,331.466667,0,0,0,0.0
8,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-12,X4,...,85.7,5323.0,15.0,354.866667,348.0,354.866667,0,0,0,0.0
9,08019301001,01,Ciutat Vella,01,el Raval,001,001,"POLYGON ((431076.9025 4581077.3095, 431058.164...",2023-01-13,X4,...,85.7,4747.0,15.0,316.466667,250.0,316.466667,0,0,0,0.0


## 8. Verify Dataset Structure

Let's verify that `gdf_daily` has the expected structure:
- 1068 census sections
- One row per census section per day from 2023-01-04 to 2024-12-31


In [43]:
# Calculate expected number of rows
date_start = pd.to_datetime('2023-01-04')
date_end = pd.to_datetime('2024-12-31')
expected_days = (date_end - date_start).days + 1  # +1 to include both start and end dates
expected_census_sections = 1068
expected_rows = expected_census_sections * expected_days

# Actual values
actual_rows = len(gdf_daily)
actual_census_sections = gdf_daily['SECCIO_CENSAL'].nunique()
actual_dates = gdf_daily['DATA_LECTURA'].nunique()
actual_date_range = (gdf_daily['DATA_LECTURA'].min(), gdf_daily['DATA_LECTURA'].max())

# Check for duplicates (should be 0)
duplicates = gdf_daily.duplicated(subset=['SECCIO_CENSAL', 'DATA_LECTURA']).sum()

print("=== Dataset Structure Verification ===\n")
print(f"Expected rows: {expected_rows:,}")
print(f"Actual rows: {actual_rows:,}")
print(f"Match: {'✓' if actual_rows == expected_rows else '✗'}\n")

print(f"Expected census sections: {expected_census_sections}")
print(f"Actual census sections: {actual_census_sections}")
print(f"Match: {'✓' if actual_census_sections == expected_census_sections else '✗'}\n")

print(f"Expected days: {expected_days}")
print(f"Actual unique dates: {actual_dates}")
print(f"Match: {'✓' if actual_dates == expected_days else '✗'}\n")

print(f"Expected date range: {date_start.date()} to {date_end.date()}")
print(f"Actual date range: {actual_date_range[0].date()} to {actual_date_range[1].date()}")
print(f"Match: {'✓' if actual_date_range[0].date() == date_start.date() and actual_date_range[1].date() == date_end.date() else '✗'}\n")

print(f"Duplicate rows (SECCIO_CENSAL + DATA_LECTURA): {duplicates}")
print(f"No duplicates: {'✓' if duplicates == 0 else '✗'}\n")

# Check if we have exactly one row per census section per date
if actual_rows == expected_rows and duplicates == 0:
    print("✓ Dataset structure is correct: one row per census section per day")
else:
    print("✗ Dataset structure needs review")
    
# Show sample of date distribution
print(f"\nSample: First few dates for one census section:")
sample_seccio = gdf_daily['SECCIO_CENSAL'].iloc[0]
print(gdf_daily[gdf_daily['SECCIO_CENSAL'] == sample_seccio][['SECCIO_CENSAL', 'DATA_LECTURA']].head(10))


=== Dataset Structure Verification ===

Expected rows: 777,504
Actual rows: 777,504
Match: ✓

Expected census sections: 1068
Actual census sections: 1068
Match: ✓

Expected days: 728
Actual unique dates: 728
Match: ✓

Expected date range: 2023-01-04 to 2024-12-31
Actual date range: 2023-01-04 to 2024-12-31
Match: ✓

Duplicate rows (SECCIO_CENSAL + DATA_LECTURA): 0
No duplicates: ✓

✓ Dataset structure is correct: one row per census section per day

Sample: First few dates for one census section:
  SECCIO_CENSAL DATA_LECTURA
0   08019301001   2023-01-04
1   08019301001   2023-01-05
2   08019301001   2023-01-06
3   08019301001   2023-01-07
4   08019301001   2023-01-08
5   08019301001   2023-01-09
6   08019301001   2023-01-10
7   08019301001   2023-01-11
8   08019301001   2023-01-12
9   08019301001   2023-01-13


## 9. Load Vulnerability Score Configuration

Load the data-driven recommendations from `correlationAnalysis.ipynb` to use recommended weather variables and weights for vulnerability score calculation.


In [44]:
import json
import numpy as np

# Load vulnerability score configuration from correlationAnalysis.ipynb
try:
    with open('vulnerability_score_config.json', 'r') as f:
        vulnerability_score_config = json.load(f)
    
    print("✓ Loaded vulnerability score configuration from correlationAnalysis.ipynb")
    if vulnerability_score_config.get('heatwave'):
        print(f"  Heatwave variables: {len(vulnerability_score_config['heatwave']['variables'])}")
    if vulnerability_score_config.get('rainfall'):
        print(f"  Rainfall variables: {len(vulnerability_score_config['rainfall']['variables'])}")
except FileNotFoundError:
    print("⚠ Warning: vulnerability_score_config.json not found.")
    print("  Please run correlationAnalysis.ipynb first to generate the configuration file.")
    print("  Falling back to default hardcoded approach.")
    vulnerability_score_config = None

# Helper function to map correlationAnalysis variable names to actual weather column names
def map_variable_to_column(var_name, weather_cols):
    """Map correlationAnalysis variable names to actual weather column names."""
    mapping = {
        'temp_max': 'Temperatura màxima diària + hora',
        'temp_avg': 'Temperatura mitjana diària',
        'temp_min': 'Temperatura mínima diària + hora',
        'temp_avg_classic': 'Temperatura mitjana diària clàssica',
        'temp_amplitude': 'Amplitud tèrmica diària',
        'solar_global': 'Irradiació solar global diària',
        'evapotranspiration': 'Evapotranspiració de referència',
        'humidity_avg': 'Humitat relativa mitjana diària',
        'humidity_max': 'Humitat relativa màxima diària + data',
        'humidity_min': 'Humitat relativa mínima diària + data',
        'rain_daily': 'Precipitació acumulada diària',
        'rain_8h': 'Precipitació acumulada diària (8-8 h)',
        'rain_max_1h': 'Precipitació màxima en 1 h (diària) + hora',
        'rain_max_30min': 'Precipitació màxima en 30 min (diària)+ hora',
        'rain_max_1min': 'Precipitació màxima en 1 min (diària) + hora',
    }
    
    if var_name in mapping:
        target = mapping[var_name]
        # Find matching column (case-insensitive, partial match)
        for col in weather_cols:
            if target.lower() in col.lower() or col.lower() in target.lower():
                return col
    return None

# Helper function to normalize weather variables to 0-1 scale
def normalize_weather_variable(series, var_type, method='percentile'):
    """
    Normalize weather variable to 0-1 scale based on type.
    
    Parameters:
    - series: pandas Series with weather values
    - var_type: string indicating variable type (e.g., 'temp_max', 'rain_daily')
    - method: 'percentile' (for temperature) or 'max' (for precipitation)
    """
    if series.isna().all() or len(series.dropna()) == 0:
        return pd.Series(0, index=series.index)
    
    # For temperature variables: use percentile-based normalization (more robust to outliers)
    if 'temp' in var_type:
        q95 = series.quantile(0.95)
        q05 = series.quantile(0.05)
        if q95 > q05:
            normalized = (series - q05) / (q95 - q05)
            return np.clip(normalized, 0, 1)
        else:
            # Fallback to min-max if percentiles are equal
            min_val = series.min()
            max_val = series.max()
            if max_val > min_val:
                normalized = (series - min_val) / (max_val - min_val)
                return np.clip(normalized, 0, 1)
    
    # For precipitation variables: normalize based on positive values
    elif 'rain' in var_type:
        max_val = series.max()
        if max_val > 0:
            normalized = series / max_val
            return np.clip(normalized, 0, 1)
    
    # For solar/evapotranspiration: standard min-max
    elif var_type in ['solar_global', 'evapotranspiration']:
        min_val = series.min()
        max_val = series.max()
        if max_val > min_val:
            normalized = (series - min_val) / (max_val - min_val)
            return np.clip(normalized, 0, 1)
    
    # For humidity: standard min-max
    elif 'humidity' in var_type:
        min_val = series.min()
        max_val = series.max()
        if max_val > min_val:
            normalized = (series - min_val) / (max_val - min_val)
            return np.clip(normalized, 0, 1)
    
    # Default: standard min-max normalization
    min_val = series.min()
    max_val = series.max()
    if max_val > min_val:
        normalized = (series - min_val) / (max_val - min_val)
        return np.clip(normalized, 0, 1)
    
    return pd.Series(0, index=series.index)


✓ Loaded vulnerability score configuration from correlationAnalysis.ipynb
  Heatwave variables: 6
  Rainfall variables: 5


### 9.1 Calculate Weather Vulnerability Using Recommended Variables

Calculate weather vulnerability components using the data-driven recommendations from `correlationAnalysis.ipynb`. This replaces the previous hardcoded approach with evidence-based variable selection and weighting.


In [45]:
# Calculate weather vulnerability using recommended variables and weights from correlationAnalysis.ipynb

# Prepare weather columns list for vulnerability calculation
# Get all weather variable columns (exclude non-weather columns)
non_weather_cols = ['SECCIO_CENSAL', 'DATA_LECTURA', 'CODI_ESTACIO', 'geometry', 'centroid', 
                    'centroid_lat', 'centroid_lon', 'dist_m', 'ist', 'CONSUMO_TOTAL', 'NUM_FUITES',
                    'codi_districte', 'nom_districte', 'codi_barri', 'nom_barri', 'codi_aeb', 
                    'codi_seccio_censal', 'NUM_CONTRACTS', 'CONSUMO_POR_CONTRATO', 'CONSUMO_PROMEDIO', 
                    'CONSUMO_MEDIANA', 'NUM_CONTRATOS_CON_FUITES', 'FUITES_MAX_POR_CONTRATO', 
                    'FUITES_PROMEDIO_POR_CONTRATO', 'DATA_LECTURA_dt', 'month', 'consum_baseline',
                    'leak_freq_mean', 'leak_contracts_mean', 'leak_max_mean', 'leak_avg_mean',
                    'consum_surge', 'contracts_surge', 'consum_intensity_surge', 'is_rainfall_event']

# Get all columns that are weather variables (from the pivot operation)
# These are columns that aren't in the non-weather list
weather_cols = [col for col in gdf_daily.columns if col not in non_weather_cols]

# Filter to only columns that contain numeric weather data (exclude text/object columns)
weather_cols = [col for col in weather_cols if gdf_daily[col].dtype in ['float64', 'int64', 'float32', 'int32']]

print(f"Prepared weather_cols: {len(weather_cols)} weather variable columns found")

# HEATWAVE WEATHER VULNERABILITY
if vulnerability_score_config and vulnerability_score_config.get('heatwave'):
    heatwave_config = vulnerability_score_config['heatwave']
    heatwave_weights = heatwave_config['weights']
    heatwave_vars = heatwave_config['variables']
    
    print("="*80)
    print("HEATWAVE WEATHER VULNERABILITY (Using Recommended Variables)")
    print("="*80)
    print(f"Recommended variables: {heatwave_vars}")
    print(f"Number of variables: {len(heatwave_vars)}\n")
    
    gdf_daily['vuln_weather_heatwave'] = 0
    total_weight = 0
    vars_used = []
    
    for var, weight in sorted(heatwave_weights.items(), key=lambda x: x[1], reverse=True):
        var_col = map_variable_to_column(var, weather_cols)
        if var_col and var_col in gdf_daily.columns:
            # Normalize the variable to 0-1 scale
            var_normalized = normalize_weather_variable(gdf_daily[var_col], var)
            gdf_daily['vuln_weather_heatwave'] += weight * var_normalized
            total_weight += weight
            vars_used.append(var)
            print(f"  ✓ {var:25s} (weight: {weight:.3f}) -> {var_col}")
        else:
            print(f"  ✗ {var:25s} -> Column not found in dataset")
    
    # Normalize to ensure weights sum to 1
    if total_weight > 0:
        gdf_daily['vuln_weather_heatwave'] = gdf_daily['vuln_weather_heatwave'] / total_weight
        print(f"\n  Used {len(vars_used)}/{len(heatwave_vars)} variables (total weight: {total_weight:.3f})")
        print(f"  Heatwave weather vulnerability: min={gdf_daily['vuln_weather_heatwave'].min():.3f}, max={gdf_daily['vuln_weather_heatwave'].max():.3f}")
    else:
        print("\n  ⚠ No variables found, using fallback")
        gdf_daily['vuln_weather_heatwave'] = 0
else:
    # Fallback to original approach if config not available
    print("⚠ Using fallback: Single temperature variable approach")
    temp_cols = [col for col in weather_cols if 'temperatura' in col.lower() or 'tèrmica' in col.lower()]
    if temp_cols:
        if any('màxima' in col.lower() for col in temp_cols):
            temp_col = [col for col in temp_cols if 'màxima' in col.lower()][0]
        else:
            temp_col = temp_cols[0]
        temp_seasonal = gdf_daily.groupby(['SECCIO_CENSAL', 'month'])[temp_col].transform('mean')
        temp_anomaly = gdf_daily[temp_col] - temp_seasonal
        temp_anomaly_max = temp_anomaly.abs().max()
        if temp_anomaly_max > 0:
            gdf_daily['vuln_weather_heatwave'] = (temp_anomaly / temp_anomaly_max).abs()
            gdf_daily['vuln_weather_heatwave'] = (gdf_daily['vuln_weather_heatwave'] - gdf_daily['vuln_weather_heatwave'].min()) / (gdf_daily['vuln_weather_heatwave'].max() - gdf_daily['vuln_weather_heatwave'].min() + 1e-10)
        else:
            gdf_daily['vuln_weather_heatwave'] = 0
    else:
        gdf_daily['vuln_weather_heatwave'] = 0

# RAINFALL WEATHER VULNERABILITY
if vulnerability_score_config and vulnerability_score_config.get('rainfall'):
    rainfall_config = vulnerability_score_config['rainfall']
    rainfall_weights = rainfall_config['weights']
    rainfall_vars = rainfall_config['variables']
    
    print("\n" + "="*80)
    print("RAINFALL WEATHER VULNERABILITY (Using Recommended Variables)")
    print("="*80)
    print(f"Recommended variables: {rainfall_vars}")
    print(f"Number of variables: {len(rainfall_vars)}\n")
    
    gdf_daily['vuln_weather_rainfall'] = 0
    total_weight = 0
    vars_used = []
    
    for var, weight in sorted(rainfall_weights.items(), key=lambda x: x[1], reverse=True):
        var_col = map_variable_to_column(var, weather_cols)
        if var_col and var_col in gdf_daily.columns:
            # Normalize the variable to 0-1 scale
            var_normalized = normalize_weather_variable(gdf_daily[var_col], var)
            gdf_daily['vuln_weather_rainfall'] += weight * var_normalized
            total_weight += weight
            vars_used.append(var)
            print(f"  ✓ {var:25s} (weight: {weight:.3f}) -> {var_col}")
        else:
            print(f"  ✗ {var:25s} -> Column not found in dataset")
    
    # Normalize to ensure weights sum to 1
    if total_weight > 0:
        gdf_daily['vuln_weather_rainfall'] = gdf_daily['vuln_weather_rainfall'] / total_weight
        print(f"\n  Used {len(vars_used)}/{len(rainfall_vars)} variables (total weight: {total_weight:.3f})")
        print(f"  Rainfall weather vulnerability: min={gdf_daily['vuln_weather_rainfall'].min():.3f}, max={gdf_daily['vuln_weather_rainfall'].max():.3f}")
    else:
        print("\n  ⚠ No variables found, using fallback")
        gdf_daily['vuln_weather_rainfall'] = 0
else:
    # Fallback to original approach if config not available
    print("\n⚠ Using fallback: Single precipitation variable approach")
    precip_cols = [col for col in weather_cols if 'precipitació' in col.lower() or 'pluja' in col.lower()]
    if precip_cols:
        precip_col = precip_cols[0]
        precip_seasonal = gdf_daily.groupby(['SECCIO_CENSAL', 'month'])[precip_col].transform('mean')
        precip_anomaly = gdf_daily[precip_col] - precip_seasonal
        precip_anomaly_max = precip_anomaly.max()
        if precip_anomaly_max > 0:
            gdf_daily['vuln_weather_rainfall'] = np.maximum(0, precip_anomaly) / precip_anomaly_max
            gdf_daily['vuln_weather_rainfall'] = (gdf_daily['vuln_weather_rainfall'] - gdf_daily['vuln_weather_rainfall'].min()) / (gdf_daily['vuln_weather_rainfall'].max() - gdf_daily['vuln_weather_rainfall'].min() + 1e-10)
        else:
            gdf_daily['vuln_weather_rainfall'] = 0
    else:
        gdf_daily['vuln_weather_rainfall'] = 0

print("\n" + "="*80)
print("✓ Weather vulnerability components calculated")
print("="*80)


Prepared weather_cols: 22 weather variable columns found
HEATWAVE WEATHER VULNERABILITY (Using Recommended Variables)
Recommended variables: ['temp_avg', 'temp_min', 'temp_avg_classic', 'temp_max', 'evapotranspiration', 'solar_global']
Number of variables: 6

  ✓ temp_avg                  (weight: 0.174) -> Temperatura mitjana diària
  ✓ temp_min                  (weight: 0.173) -> Temperatura mínima diària + hora
  ✓ temp_avg_classic          (weight: 0.172) -> Temperatura mitjana diària
  ✓ temp_max                  (weight: 0.166) -> Temperatura màxima diària + hora
  ✓ evapotranspiration        (weight: 0.161) -> Evapotranspiració de referència
  ✓ solar_global              (weight: 0.155) -> Irradiació solar global diària

  Used 6/6 variables (total weight: 1.001)
  Heatwave weather vulnerability: min=0.001, max=0.981

RAINFALL WEATHER VULNERABILITY (Using Recommended Variables)
Recommended variables: ['rain_max_30min', 'rain_max_1h', 'rain_max_1min', 'rain_daily', 'rain_8h']
Num

In [46]:
# Calculate infrastructure and socioeconomic vulnerability components

print("="*80)
print("INFRASTRUCTURE & SOCIOECONOMIC VULNERABILITY")
print("="*80)
print("Using comprehensive aggregation metrics from sections 7.2 and 7.3")
print("="*80)

# 1. INFRASTRUCTURE VULNERABILITY
# Create separate infrastructure vulnerability components for heatwave and rainfall
# to better capture how infrastructure responds to different weather conditions
# Using all aggregation metrics: consumption (density, intensity) and leaks (density, severity, intensity)

# === COMMON BASELINE METRICS (Using Multiple Leak Metrics) ===
# Historical leak baseline vulnerability - composite metric using all leak aggregation metrics
# Fill missing leak metrics with 0 for baseline calculation
leak_cols = ['NUM_FUITES', 'NUM_CONTRATOS_CON_FUITES', 'FUITES_MAX_POR_CONTRATO', 'FUITES_PROMEDIO_POR_CONTRATO']
for col in leak_cols:
    if col not in gdf_daily.columns:
        gdf_daily[col] = 0.0
    gdf_daily[col] = gdf_daily[col].fillna(0.0)

# Calculate baseline leak metrics (historical averages per census section)
leak_baselines = gdf_daily.groupby('SECCIO_CENSAL', as_index=False).agg({
    'NUM_FUITES': 'mean',                    # Average total leaks
    'NUM_CONTRATOS_CON_FUITES': 'mean',      # Average contracts with leaks
    'FUITES_MAX_POR_CONTRATO': 'mean',       # Average max severity
    'FUITES_PROMEDIO_POR_CONTRATO': 'mean'   # Average intensity
})
leak_baselines.columns = ['SECCIO_CENSAL', 'leak_freq_mean', 'leak_contracts_mean', 'leak_max_mean', 'leak_avg_mean']
gdf_daily = gdf_daily.merge(leak_baselines, on='SECCIO_CENSAL', how='left')

# Normalize each baseline leak metric to 0-1 scale
for metric, col_name in [('leak_freq_mean', 'vuln_infra_leak_freq'),
                          ('leak_contracts_mean', 'vuln_infra_leak_contracts'),
                          ('leak_max_mean', 'vuln_infra_leak_max'),
                          ('leak_avg_mean', 'vuln_infra_leak_avg')]:
    if gdf_daily[metric].max() > gdf_daily[metric].min():
        gdf_daily[col_name] = (
            (gdf_daily[metric] - gdf_daily[metric].min()) / 
            (gdf_daily[metric].max() - gdf_daily[metric].min())
        )
    else:
        gdf_daily[col_name] = 0.0

# Combined baseline leak vulnerability (weighted combination of all metrics)
# 40% frequency, 30% density, 20% severity, 10% intensity
gdf_daily['vuln_infra_leaks_baseline'] = (
    0.40 * gdf_daily['vuln_infra_leak_freq'] +
    0.30 * gdf_daily['vuln_infra_leak_contracts'] +
    0.20 * gdf_daily['vuln_infra_leak_max'] +
    0.10 * gdf_daily['vuln_infra_leak_avg']
)

# Historical consumption baseline (for calculating consumption surge during heatwaves)
# Keep NaN values for missing consumption data - sections without consumption data will have NaN vulnerability scores
consum_cols = ['CONSUMO_TOTAL', 'NUM_CONTRACTS', 'CONSUMO_POR_CONTRATO']
for col in consum_cols:
    if col not in gdf_daily.columns:
        gdf_daily[col] = np.nan
    # Do NOT fill NaN - missing consumption data is meaningful and should propagate to vulnerability scores

gdf_daily['DATA_LECTURA_dt'] = pd.to_datetime(gdf_daily['DATA_LECTURA'])
gdf_daily['month'] = gdf_daily['DATA_LECTURA_dt'].dt.month

# Calculate monthly baseline for all consumption metrics
gdf_daily['consum_baseline'] = gdf_daily.groupby(['SECCIO_CENSAL', 'month'])['CONSUMO_TOTAL'].transform('mean')
gdf_daily['contracts_baseline'] = gdf_daily.groupby(['SECCIO_CENSAL', 'month'])['NUM_CONTRACTS'].transform('mean')
gdf_daily['consum_per_contract_baseline'] = gdf_daily.groupby(['SECCIO_CENSAL', 'month'])['CONSUMO_POR_CONTRATO'].transform('mean')

# === HEATWAVE INFRASTRUCTURE VULNERABILITY ===
# For heatwaves: infrastructure stress comes from consumption surge + baseline leak frequency
# Using multiple consumption metrics: total surge, contract density, and consumption intensity
if 'CONSUMO_TOTAL' in gdf_daily.columns:
    # 1. Total consumption surge (demand level)
    gdf_daily['consum_surge'] = gdf_daily['CONSUMO_TOTAL'] - gdf_daily['consum_baseline']
    # Keep NaN - do not fill with 0 (missing consumption data should propagate)
    
    # 2. Contract density surge (more contracts = more infrastructure to maintain)
    gdf_daily['contracts_surge'] = gdf_daily['NUM_CONTRACTS'] - gdf_daily['contracts_baseline'].replace(0, np.nan)
    # Keep NaN - do not fill with 0 (missing consumption data should propagate)
    
    # 3. Consumption intensity surge (higher per-contract usage = more stress)
    gdf_daily['consum_intensity_surge'] = gdf_daily['CONSUMO_POR_CONTRATO'] - gdf_daily['consum_per_contract_baseline']
    # Keep NaN - do not fill with 0 (missing consumption data should propagate)
    
    # Normalize each consumption surge metric to 0-1 scale
    # Note: NaN values will remain NaN (sections without consumption data)
    for metric, col_name in [('consum_surge', 'vuln_infra_consum_surge'),
                              ('contracts_surge', 'vuln_infra_contracts_surge'),
                              ('consum_intensity_surge', 'vuln_infra_consum_intensity_surge')]:
        # Only normalize non-NaN values
        metric_valid = gdf_daily[metric].dropna()
        if len(metric_valid) > 0:
            metric_max = metric_valid.max()
            metric_min = metric_valid.min()
            if metric_max > metric_min:
                # Normalize to 0-1, where higher surge = higher vulnerability
                # NaN values remain NaN
                gdf_daily[col_name] = (gdf_daily[metric] - metric_min) / (metric_max - metric_min)
                gdf_daily[col_name] = np.clip(gdf_daily[col_name], 0, 1)
            else:
                # All values are the same (or all NaN) - set to 0 for non-NaN, keep NaN for NaN
                gdf_daily[col_name] = gdf_daily[metric].where(pd.isna(gdf_daily[metric]), 0.0)
        else:
            # All values are NaN - keep as NaN
            gdf_daily[col_name] = gdf_daily[metric]
    
    # Combined consumption surge vulnerability (weighted combination)
    # 50% total surge, 30% density, 20% intensity
    # If any component is NaN, the result will be NaN (sections without consumption data)
    gdf_daily['vuln_infra_consumption_surge'] = (
        0.50 * gdf_daily['vuln_infra_consum_surge'] +
        0.30 * gdf_daily['vuln_infra_contracts_surge'] +
        0.20 * gdf_daily['vuln_infra_consum_intensity_surge']
    )
    
    # Heatwave infrastructure: 60% consumption surge + 40% baseline leak frequency
    # If consumption surge is NaN, the result will be NaN (sections without consumption data)
    gdf_daily['vuln_infrastructure_heatwave'] = (
        0.6 * gdf_daily['vuln_infra_consumption_surge'] + 
        0.4 * gdf_daily['vuln_infra_leaks_baseline']
    )
else:
    gdf_daily['vuln_infra_consumption_surge'] = 0.0
    gdf_daily['vuln_infrastructure_heatwave'] = gdf_daily['vuln_infra_leaks_baseline']

# === RAINFALL INFRASTRUCTURE VULNERABILITY ===
# For rainfall: infrastructure stress comes from leak response to rain + baseline leak frequency
# Using multiple leak metrics: total response, contract density response, severity response, intensity response

# Get rainfall indicator (using daily precipitation if available)
rainfall_cols = [col for col in gdf_daily.columns if 'precipitació' in col.lower() or 'pluja' in col.lower() or 'rain' in col.lower()]
if rainfall_cols:
    rain_col = rainfall_cols[0]
    gdf_daily['is_rainfall_event'] = (gdf_daily[rain_col] > 5.0).astype(int)
    
    # Calculate leak response metrics: compare rainy days vs non-rainy days for each metric
    leak_metrics_response = {}
    for metric in ['NUM_FUITES', 'NUM_CONTRATOS_CON_FUITES', 'FUITES_MAX_POR_CONTRATO', 'FUITES_PROMEDIO_POR_CONTRATO']:
        leak_on_rain = gdf_daily[gdf_daily['is_rainfall_event'] == 1].groupby('SECCIO_CENSAL')[metric].mean()
        leak_on_no_rain = gdf_daily[gdf_daily['is_rainfall_event'] == 0].groupby('SECCIO_CENSAL')[metric].mean()
        leak_response = (leak_on_rain - leak_on_no_rain).fillna(0)
        leak_metrics_response[metric] = leak_response.reset_index()
        leak_metrics_response[metric].columns = ['SECCIO_CENSAL', f'leak_response_{metric}']
        gdf_daily = gdf_daily.merge(leak_metrics_response[metric], on='SECCIO_CENSAL', how='left')
    
    # Normalize each leak response metric to 0-1 scale
    response_cols = {
        'leak_response_NUM_FUITES': 'vuln_infra_leak_response_freq',
        'leak_response_NUM_CONTRATOS_CON_FUITES': 'vuln_infra_leak_response_contracts',
        'leak_response_FUITES_MAX_POR_CONTRATO': 'vuln_infra_leak_response_max',
        'leak_response_FUITES_PROMEDIO_POR_CONTRATO': 'vuln_infra_leak_response_avg'
    }
    
    for response_col, vuln_col in response_cols.items():
        if response_col in gdf_daily.columns:
            if gdf_daily[response_col].max() > gdf_daily[response_col].min():
                gdf_daily[vuln_col] = (
                    (gdf_daily[response_col] - gdf_daily[response_col].min()) / 
                    (gdf_daily[response_col].max() - gdf_daily[response_col].min())
                )
            else:
                gdf_daily[vuln_col] = 0.0
        else:
            gdf_daily[vuln_col] = 0.0
    
    # Combined leak response vulnerability (weighted combination)
    # 40% frequency response, 30% density response, 20% severity response, 10% intensity response
    gdf_daily['vuln_infra_leak_response'] = (
        0.40 * gdf_daily.get('vuln_infra_leak_response_freq', 0) +
        0.30 * gdf_daily.get('vuln_infra_leak_response_contracts', 0) +
        0.20 * gdf_daily.get('vuln_infra_leak_response_max', 0) +
        0.10 * gdf_daily.get('vuln_infra_leak_response_avg', 0)
    )
    
    # Rainfall infrastructure: 60% leak response to rainfall + 40% baseline leak frequency
    gdf_daily['vuln_infrastructure_rainfall'] = (
        0.6 * gdf_daily['vuln_infra_leak_response'] + 
        0.4 * gdf_daily['vuln_infra_leaks_baseline']
    )
else:
    gdf_daily['vuln_infra_leak_response'] = 0.0
    gdf_daily['vuln_infrastructure_rainfall'] = gdf_daily['vuln_infra_leaks_baseline']

# Keep a general infrastructure vulnerability for backwards compatibility (average of both)
gdf_daily['vuln_infrastructure'] = 0.5 * gdf_daily['vuln_infrastructure_heatwave'] + 0.5 * gdf_daily['vuln_infrastructure_rainfall']

print("\nInfrastructure Vulnerability (using comprehensive metrics):")
print(f"  Baseline leak frequency (composite): min={gdf_daily['vuln_infra_leaks_baseline'].min():.3f}, max={gdf_daily['vuln_infra_leaks_baseline'].max():.3f}")
print(f"  Heatwave (consumption surge - composite): min={gdf_daily['vuln_infra_consumption_surge'].min():.3f}, max={gdf_daily['vuln_infra_consumption_surge'].max():.3f}")
print(f"  Heatwave infrastructure: min={gdf_daily['vuln_infrastructure_heatwave'].min():.3f}, max={gdf_daily['vuln_infrastructure_heatwave'].max():.3f}")
if 'vuln_infra_leak_response' in gdf_daily.columns:
    print(f"  Rainfall (leak response - composite): min={gdf_daily['vuln_infra_leak_response'].min():.3f}, max={gdf_daily['vuln_infra_leak_response'].max():.3f}")
print(f"  Rainfall infrastructure: min={gdf_daily['vuln_infrastructure_rainfall'].min():.3f}, max={gdf_daily['vuln_infrastructure_rainfall'].max():.3f}")

# 2. SOCIOECONOMIC VULNERABILITY
# Based on IST (Índex socioeconòmic territorial)
# Lower IST = higher vulnerability (inverse relationship)
# IST typically ranges from ~50 to ~140, where lower values indicate more vulnerable areas

if 'ist' in gdf_daily.columns and gdf_daily['ist'].notna().any():
    # Normalize IST to vulnerability (inverse: lower IST = higher vulnerability)
    ist_min = gdf_daily['ist'].min()
    ist_max = gdf_daily['ist'].max()
    
    if ist_max > ist_min:
        # Inverse normalization: (max - value) / (max - min)
        # This makes lower IST values (more vulnerable) map to higher vulnerability scores
        gdf_daily['vuln_socioeconomic'] = (ist_max - gdf_daily['ist']) / (ist_max - ist_min)
    else:
        gdf_daily['vuln_socioeconomic'] = 0.0
    
    print("\nSocioeconomic Vulnerability (from IST):")
    print(f"  IST range: {ist_min:.1f} to {ist_max:.1f}")
    print(f"  Vulnerability: min={gdf_daily['vuln_socioeconomic'].min():.3f}, max={gdf_daily['vuln_socioeconomic'].max():.3f}")
    print(f"  (Lower IST → Higher vulnerability)")
else:
    print("\n⚠ IST not available, setting socioeconomic vulnerability to 0")
    gdf_daily['vuln_socioeconomic'] = 0.0

print("\n" + "="*80)
print("✓ Infrastructure and socioeconomic vulnerability components calculated")
print("="*80)


INFRASTRUCTURE & SOCIOECONOMIC VULNERABILITY
Using comprehensive aggregation metrics from sections 7.2 and 7.3

Infrastructure Vulnerability (using comprehensive metrics):
  Baseline leak frequency (composite): min=0.000, max=1.000
  Heatwave (consumption surge - composite): min=0.045, max=0.847
  Heatwave infrastructure: min=0.186, max=0.714
  Rainfall (leak response - composite): min=0.000, max=1.000
  Rainfall infrastructure: min=0.254, max=0.897

Socioeconomic Vulnerability (from IST):
  IST range: 51.7 to 138.0
  Vulnerability: min=0.000, max=1.000
  (Lower IST → Higher vulnerability)

✓ Infrastructure and socioeconomic vulnerability components calculated


### 9.3 Calculate Final Vulnerability Scores

Combine weather, infrastructure, and socioeconomic components to create final vulnerability scores for heatwave and rainfall.

**Weighting scheme:**
- **Weather vulnerability**: 60% (primary driver, varies daily)
- **Infrastructure vulnerability**: 25% (static factor, reflects historical infrastructure condition)
- **Socioeconomic vulnerability**: 15% (static factor, reflects social vulnerability)

This weighting emphasizes weather conditions while accounting for infrastructure and social factors.


In [47]:
# Combine all vulnerability components into final scores

print("="*80)
print("FINAL VULNERABILITY SCORE CALCULATION")
print("="*80)

# Define weights for combining components
WEIGHT_WEATHER = 0.60         # 60% - weather conditions (varies daily)
WEIGHT_INFRASTRUCTURE = 0.25  # 25% - infrastructure factors (varies daily for heatwave/rainfall)
WEIGHT_SOCIOECONOMIC = 0.15   # 15% - socioeconomic factors (static)

print(f"\nComponent weights:")
print(f"  Weather:        {WEIGHT_WEATHER*100:.0f}%")
print(f"  Infrastructure: {WEIGHT_INFRASTRUCTURE*100:.0f}%")
print(f"  Socioeconomic:  {WEIGHT_SOCIOECONOMIC*100:.0f}%")
print(f"  Total:          {(WEIGHT_WEATHER + WEIGHT_INFRASTRUCTURE + WEIGHT_SOCIOECONOMIC)*100:.0f}%")

# Ensure all components exist and are normalized (0-1 scale)
# Weather components should already be normalized from previous steps
if 'vuln_weather_heatwave' not in gdf_daily.columns:
    print("\n⚠ vuln_weather_heatwave not found, setting to 0")
    gdf_daily['vuln_weather_heatwave'] = 0.0

if 'vuln_weather_rainfall' not in gdf_daily.columns:
    print("⚠ vuln_weather_rainfall not found, setting to 0")
    gdf_daily['vuln_weather_rainfall'] = 0.0

if 'vuln_infrastructure_heatwave' not in gdf_daily.columns:
    print("⚠ vuln_infrastructure_heatwave not found, setting to 0")
    gdf_daily['vuln_infrastructure_heatwave'] = 0.0

if 'vuln_infrastructure_rainfall' not in gdf_daily.columns:
    print("⚠ vuln_infrastructure_rainfall not found, setting to 0")
    gdf_daily['vuln_infrastructure_rainfall'] = 0.0

if 'vuln_socioeconomic' not in gdf_daily.columns:
    print("⚠ vuln_socioeconomic not found, setting to 0")
    gdf_daily['vuln_socioeconomic'] = 0.0

# Fill NaN values with 0 for weather and socioeconomic (these should always have values)
# Do NOT fill infrastructure NaN - missing consumption data should result in NaN vulnerability scores
gdf_daily['vuln_weather_heatwave'] = gdf_daily['vuln_weather_heatwave'].fillna(0.0)
gdf_daily['vuln_weather_rainfall'] = gdf_daily['vuln_weather_rainfall'].fillna(0.0)
# Keep NaN for infrastructure components - sections without consumption data will have NaN infrastructure vulnerability
# gdf_daily['vuln_infrastructure_heatwave'] = gdf_daily['vuln_infrastructure_heatwave'].fillna(0.0)  # REMOVED: preserve NaN
# gdf_daily['vuln_infrastructure_rainfall'] = gdf_daily['vuln_infrastructure_rainfall'].fillna(0.0)  # REMOVED: preserve NaN
gdf_daily['vuln_socioeconomic'] = gdf_daily['vuln_socioeconomic'].fillna(0.0)

# Calculate final vulnerability scores
# HEATWAVE VULNERABILITY SCORE
gdf_daily['VULNERABILITY_SCORE_HEATWAVE'] = (
    WEIGHT_WEATHER * gdf_daily['vuln_weather_heatwave'] +
    WEIGHT_INFRASTRUCTURE * gdf_daily['vuln_infrastructure_heatwave'] +
    WEIGHT_SOCIOECONOMIC * gdf_daily['vuln_socioeconomic']
)

# RAINFALL VULNERABILITY SCORE
gdf_daily['VULNERABILITY_SCORE_RAINFALL'] = (
    WEIGHT_WEATHER * gdf_daily['vuln_weather_rainfall'] +
    WEIGHT_INFRASTRUCTURE * gdf_daily['vuln_infrastructure_rainfall'] +
    WEIGHT_SOCIOECONOMIC * gdf_daily['vuln_socioeconomic']
)

# Ensure scores are in 0-1 range (should already be, but clip to be safe)
gdf_daily['VULNERABILITY_SCORE_HEATWAVE'] = np.clip(gdf_daily['VULNERABILITY_SCORE_HEATWAVE'], 0, 1)
gdf_daily['VULNERABILITY_SCORE_RAINFALL'] = np.clip(gdf_daily['VULNERABILITY_SCORE_RAINFALL'], 0, 1)

print("\n" + "="*80)
print("FINAL VULNERABILITY SCORES")
print("="*80)
print(f"\nHeatwave Vulnerability Score:")
print(f"  Min: {gdf_daily['VULNERABILITY_SCORE_HEATWAVE'].min():.4f}")
print(f"  Max: {gdf_daily['VULNERABILITY_SCORE_HEATWAVE'].max():.4f}")
print(f"  Mean: {gdf_daily['VULNERABILITY_SCORE_HEATWAVE'].mean():.4f}")
print(f"  Median: {gdf_daily['VULNERABILITY_SCORE_HEATWAVE'].median():.4f}")

print(f"\nRainfall Vulnerability Score:")
print(f"  Min: {gdf_daily['VULNERABILITY_SCORE_RAINFALL'].min():.4f}")
print(f"  Max: {gdf_daily['VULNERABILITY_SCORE_RAINFALL'].max():.4f}")
print(f"  Mean: {gdf_daily['VULNERABILITY_SCORE_RAINFALL'].mean():.4f}")
print(f"  Median: {gdf_daily['VULNERABILITY_SCORE_RAINFALL'].median():.4f}")

# Show sample of final scores
print(f"\nSample of final vulnerability scores (first 10 rows):")
sample_cols = ['SECCIO_CENSAL', 'DATA_LECTURA', 'VULNERABILITY_SCORE_HEATWAVE', 'VULNERABILITY_SCORE_RAINFALL']
print(gdf_daily[sample_cols].head(10))

print("\n" + "="*80)
print("✓ Final vulnerability scores calculated successfully")
print("="*80)


FINAL VULNERABILITY SCORE CALCULATION

Component weights:
  Weather:        60%
  Infrastructure: 25%
  Socioeconomic:  15%
  Total:          100%

FINAL VULNERABILITY SCORES

Heatwave Vulnerability Score:
  Min: 0.0688
  Max: 0.8147
  Mean: 0.4030
  Median: 0.3904

Rainfall Vulnerability Score:
  Min: 0.0644
  Max: 0.7294
  Mean: 0.1302
  Median: 0.1181

Sample of final vulnerability scores (first 10 rows):
  SECCIO_CENSAL DATA_LECTURA  VULNERABILITY_SCORE_HEATWAVE  \
0   08019301001   2023-01-04                      0.279945   
1   08019301001   2023-01-05                      0.270259   
2   08019301001   2023-01-06                      0.249357   
3   08019301001   2023-01-07                      0.273828   
4   08019301001   2023-01-08                      0.275974   
5   08019301001   2023-01-09                      0.316919   
6   08019301001   2023-01-10                      0.263755   
7   08019301001   2023-01-11                      0.264968   
8   08019301001   2023-01-12  

## 10. Save Results

Save the final dataset with vulnerability scores for use in mapping and analysis.


In [48]:
# Save the dataset with vulnerability scores
print("="*80)
print("SAVING RESULTS")
print("="*80)

# Ensure output directory exists
import os
output_dir = "clean"
os.makedirs(output_dir, exist_ok=True)

# Save as parquet (recommended for large datasets with geometry)
# GeoPandas to_parquet() automatically handles geometry column
output_file = os.path.join(output_dir, "vulnerability_daily.parquet")

print(f"\nSaving dataset to: {output_file}")
print(f"Dataset shape: {gdf_daily.shape} (rows, columns)")

# Identify key columns to save (all columns will be saved, but let's list important ones)
key_columns = [
    'SECCIO_CENSAL', 'DATA_LECTURA', 'geometry',
    'VULNERABILITY_SCORE_HEATWAVE', 'VULNERABILITY_SCORE_RAINFALL',
    'vuln_weather_heatwave', 'vuln_weather_rainfall',
    'vuln_infrastructure_heatwave', 'vuln_infrastructure_rainfall',
    'vuln_socioeconomic'
]

# Check which key columns exist
existing_key_cols = [col for col in key_columns if col in gdf_daily.columns]
missing_key_cols = [col for col in key_columns if col not in gdf_daily.columns]

print(f"\nKey columns being saved ({len(existing_key_cols)}/{len(key_columns)}):")
for col in existing_key_cols[:10]:  # Show first 10
    print(f"  ✓ {col}")
if len(existing_key_cols) > 10:
    print(f"  ... and {len(existing_key_cols) - 10} more columns")

if missing_key_cols:
    print(f"\n⚠ Missing key columns: {missing_key_cols}")

# Save the GeoDataFrame
# GeoPandas to_parquet() preserves geometry automatically
try:
    gdf_daily.to_parquet(output_file, index=False)
    print(f"\n✓ Successfully saved dataset to: {output_file}")
    
    # Verify the file was created and get its size
    file_size = os.path.getsize(output_file) / (1024 * 1024)  # Size in MB
    print(f"  File size: {file_size:.2f} MB")
    
except Exception as e:
    print(f"\n✗ Error saving dataset: {e}")
    raise

# Summary statistics
print(f"\n" + "="*80)
print("DATASET SUMMARY")
print("="*80)
print(f"Total rows: {len(gdf_daily):,} (census sections × days)")
print(f"Total columns: {len(gdf_daily.columns)}")
print(f"Date range: {gdf_daily['DATA_LECTURA'].min().date()} to {gdf_daily['DATA_LECTURA'].max().date()}")
print(f"Census sections: {gdf_daily['SECCIO_CENSAL'].nunique():,}")
print(f"Unique dates: {gdf_daily['DATA_LECTURA'].nunique()}")

print(f"\nFinal Vulnerability Scores:")
print(f"  Heatwave: min={gdf_daily['VULNERABILITY_SCORE_HEATWAVE'].min():.4f}, "
      f"max={gdf_daily['VULNERABILITY_SCORE_HEATWAVE'].max():.4f}, "
      f"mean={gdf_daily['VULNERABILITY_SCORE_HEATWAVE'].mean():.4f}")
print(f"  Rainfall: min={gdf_daily['VULNERABILITY_SCORE_RAINFALL'].min():.4f}, "
      f"max={gdf_daily['VULNERABILITY_SCORE_RAINFALL'].max():.4f}, "
      f"mean={gdf_daily['VULNERABILITY_SCORE_RAINFALL'].mean():.4f}")

# Check geometry is preserved
if 'geometry' in gdf_daily.columns:
    geom_type = gdf_daily.geometry.geom_type.unique()
    print(f"\nGeometry information:")
    print(f"  Geometry column: ✓ present")
    print(f"  Geometry types: {', '.join(geom_type)}")
    print(f"  CRS: {gdf_daily.crs}")

print("\n" + "="*80)
print("✓ Save complete")
print("="*80)


SAVING RESULTS

Saving dataset to: clean/vulnerability_daily.parquet
Dataset shape: (777504, 86) (rows, columns)

Key columns being saved (10/10):
  ✓ SECCIO_CENSAL
  ✓ DATA_LECTURA
  ✓ geometry
  ✓ VULNERABILITY_SCORE_HEATWAVE
  ✓ VULNERABILITY_SCORE_RAINFALL
  ✓ vuln_weather_heatwave
  ✓ vuln_weather_rainfall
  ✓ vuln_infrastructure_heatwave
  ✓ vuln_infrastructure_rainfall
  ✓ vuln_socioeconomic

✓ Successfully saved dataset to: clean/vulnerability_daily.parquet
  File size: 36.54 MB

DATASET SUMMARY
Total rows: 777,504 (census sections × days)
Total columns: 86
Date range: 2023-01-04 to 2024-12-31
Census sections: 1,068
Unique dates: 728

Final Vulnerability Scores:
  Heatwave: min=0.0688, max=0.8147, mean=0.4030
  Rainfall: min=0.0644, max=0.7294, mean=0.1302

Geometry information:
  Geometry column: ✓ present
  Geometry types: Polygon, MultiPolygon
  CRS: EPSG:4326

✓ Save complete
